In [1]:
# Importando las librerías necesarias
import pandas as pd
import numpy as np
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
# Importando las librerías para la selección de características
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
import optuna

/Users/cash/Desktop/F5/Python/Machine_Learning/non-supervised-learning/env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Cargar los datos de entrenamiento y prueba
data = pd.read_csv('train.csv')

In [4]:
data.shape

(8693, 14)

In [5]:
data.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [6]:
# Data preprocessing

# Drop Rows with Missing Values
data.fillna(0, inplace=True)

# Drop the Passenger ID Column as its the Unique Identifier
data.drop('PassengerId', inplace=True, axis=1)

# Convert mixed type columns to string
for col in data.columns:
    if data[col].dtype == 'object':
        data[col] = data[col].astype(str)

# Identify categorical columns
categorical_cols = [col for col in data.columns if data[col].dtype == 'object']

# Apply label encoding to categorical columns
oe = {}
for col in categorical_cols:
    oe[col] = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
    data[col] = oe[col].fit_transform(data[col].values.reshape(-1, 1))

# Separating features and target
X = data.drop('Transported', axis=1)
y = data['Transported']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
(X_train.shape, y_train.shape), (X_test.shape, y_test.shape)

(((6954, 12), (6954,)), ((1739, 12), (1739,)))

In [8]:
def objective(trial):
    """
    Defining the objective function for Optuna with feature selection
    """
    n_estimators = trial.suggest_int('n_estimators', 100, 1000)
    max_depth = trial.suggest_int('max_depth', 2, 32)
    # Number of features to select
    k_best = trial.suggest_int('k_best', 1, X_train.shape[1])  

    # Feature selection
    select_k_best = SelectKBest(score_func=f_classif, k=k_best) # SelectKBest: Esta clase se utiliza para seleccionar las k mejores características según un criterio de puntuación. En este caso, f_classif se utiliza como la función de puntuación, que es adecuada para clasificación.
    preprocessor = ColumnTransformer([('select', select_k_best, X.columns)], remainder='passthrough') # Se define un transformador de columnas que aplica la selección de características a las columnas especificadas en X.columns. El parámetro remainder='passthrough' asegura que las columnas no seleccionadas se mantengan sin cambios.
    
    # Building the model pipeline
    clf = Pipeline(steps=[('preprocessor', preprocessor),
                          ('classifier', RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=42))]) # Se crea un pipeline que encadena el preprocesador y el clasificador. Esto simplifica el proceso de entrenamiento y predicción, asegurando que las transformaciones se apliquen correctamente. RandomForestClassifier: Se inicializa el clasificador con los hiperparámetros sugeridos (número de estimadores y profundidad máxima).

    clf.fit(X_train, y_train) # Se entrena el modelo (clf) usando los datos de entrenamiento (X_train, y_train).
    y_pred = clf.predict(X_test) # Se realizan predicciones sobre el conjunto de prueba (X_test).
    accuracy = accuracy_score(y_test, y_pred) # Se calcula la precisión del modelo utilizando la función accuracy_score, comparando las predicciones (y_pred) con las etiquetas verdaderas (y_test).
    
    # Storing the trained classifier in the study object
    trial.set_user_attr('classifier', clf) # almacena el clasificador entrenado en el objeto trial. Esto puede ser útil para análisis posteriores o para evaluar el rendimiento del modelo después de la optimización
    return accuracy

In [9]:
# Running the optimization
study = optuna.create_study(study_name='best_features_and_hyperparameters', direction='maximize') # optuna.create_study(...) es una función que se utiliza para crear un nuevo objeto de estudio. Un estudio en Optuna es un contenedor para todas las pruebas (trials) que se realizarán durante la optimización. Tiene dos parámetros: 1. study_name='best_features_and_hyperparameters': Este es el nombre que se le asigna al estudio. Puedes usar este nombre para identificar el estudio en la base de datos de Optuna, especialmente si estás guardando resultados o usando la interfaz de visualización. 2. direction='maximize': Este parámetro indica que el objetivo de la optimización es maximizar el valor que se devolverá en la función objetivo (en este caso, la precisión del modelo). Si en lugar de maximizar quisieras minimizar un valor (como la pérdida), usarías direction='minimize'.
study.optimize(objective, n_trials=50) # study.optimize(...): es un método que ejecuta el proceso de optimización, llamando repetidamente a la función objetivo para encontrar la mejor combinación de hiperparámetros y características. Tiene dos parámetros: 1. objective: Este es el nombre de la función objetivo que se definió anteriormente. Optuna la llamará múltiples veces con diferentes configuraciones de hiperparámetros, que se generan automáticamente. 2. n_trials=50: Este parámetro indica el número total de pruebas (trials) que se realizarán. En cada prueba, Optuna selecciona un conjunto diferente de hiperparámetros y características que serán probados en el modelo. En este caso, se realizarán 50 pruebas diferentes.

[I 2024-09-23 12:47:44,271] A new study created in memory with name: best_features_and_hyperparameters
[I 2024-09-23 12:47:49,218] Trial 0 finished with value: 0.7659574468085106 and parameters: {'n_estimators': 393, 'max_depth': 28, 'k_best': 10}. Best is trial 0 with value: 0.7659574468085106.
[I 2024-09-23 12:47:50,447] Trial 1 finished with value: 0.7561817136285222 and parameters: {'n_estimators': 250, 'max_depth': 3, 'k_best': 11}. Best is trial 0 with value: 0.7659574468085106.
[I 2024-09-23 12:47:54,725] Trial 2 finished with value: 0.7625071880391029 and parameters: {'n_estimators': 506, 'max_depth': 18, 'k_best': 5}. Best is trial 0 with value: 0.7659574468085106.
[I 2024-09-23 12:48:03,018] Trial 3 finished with value: 0.7780333525014376 and parameters: {'n_estimators': 620, 'max_depth': 18, 'k_best': 12}. Best is trial 3 with value: 0.7780333525014376.
[I 2024-09-23 12:48:07,126] Trial 4 finished with value: 0.7613571017826337 and parameters: {'n_estimators': 909, 'max_dept

In [10]:
# Extracting the best hyperparameters and selected features
print('Best hyperparameters: ', study.best_params) # study.best_params: Este atributo contiene un diccionario con los mejores hiperparámetros que Optuna encontró durante el proceso de optimización. Los hiperparámetros se refieren a los valores óptimos que maximizan el rendimiento del modelo según la función objetivo definida.
best_trial = study.best_trial # study.best_trial: Este atributo proporciona acceso al trial (prueba) que produjo el mejor resultado durante la optimización. Incluye información sobre los hiperparámetros utilizados, el valor de la función objetivo (por ejemplo, la precisión del modelo), y otros metadatos relevantes.
selected_features = SelectKBest(score_func=f_classif, 
                                k=best_trial.params['k_best']).fit(X_train, y_train) # SelectKBest: Esta clase se utiliza para seleccionar las k mejores características según una función de puntuación. En este caso, f_classif es la función de puntuación que evalúa la relación entre las características y la variable objetivo.

                                # SelectKBest: Esta clase se utiliza para seleccionar las k mejores características según una función de puntuación. En este caso, f_classif es la función de puntuación que evalúa la relación entre las características y la variable objetivo.
                                
                                # k=best_trial.params['k_best']: Aquí, se utiliza el valor de k_best que se encontró durante la optimización como el número de características a seleccionar. Este valor se extrae del mejor trial.
                                
                                # .fit(X_train, y_train): Este método ajusta el selector de características a los datos de entrenamiento, calculando las puntuaciones de las características y seleccionando las mejores.

print('Selected features: ', X.columns[selected_features.get_support()]) # selected_features.get_support(): Este método devuelve un array booleano que indica qué características han sido seleccionadas por el SelectKBest. Devuelve True para las características seleccionadas y False para las que no lo son.

# X.columns[...]: Aquí se utilizan los índices booleanos para filtrar las columnas de X. Esto devuelve solo las columnas (características) que han sido seleccionadas.

Best hyperparameters:  {'n_estimators': 848, 'max_depth': 10, 'k_best': 12}
Selected features:  Index(['HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'Age', 'VIP',
       'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'Name'],
      dtype='object')


In [12]:
# Load Submission Data
submission_data = pd.read_csv('test.csv')

# Drop Rows with Missing Values
submission_data.fillna(0, inplace=True)

# Applying the label encoders to the submission data
for column, encoder in oe.items():
    submission_data[column] = encoder.transform(submission_data[column].values.reshape(-1, 1))
    
# Generate Sample Submission
best_clf = best_trial.user_attrs['classifier']
y_pred = best_clf.predict(submission_data)
submission_data['Transported'] = y_pred

# Only keep required columns for submission
submission_data = submission_data[['PassengerId', 'Transported']]

# Save the DataFrame to Kaggle Directory and Upload to Submit
submission_data.to_csv('submission.csv', index=False, header=True)